In [36]:
!pip install --upgrade tensorflow

    100% |████████████████████████████████| 45.9MB 12kB/s  eta 0:00:01
    100% |████████████████████████████████| 3.1MB 198kB/s eta 0:00:01
    100% |████████████████████████████████| 12.1MB 48kB/s eta 0:00:011
Requirement already up-to-date: mock>=2.0.0 in /usr/local/envs/py2env/lib/python2.7/site-packages (from tensorflow)
Requirement already up-to-date: wheel in /usr/local/envs/py2env/lib/python2.7/site-packages (from tensorflow)
Requirement already up-to-date: backports.weakref>=1.0rc1 in /usr/local/envs/py2env/lib/python2.7/site-packages (from tensorflow)
Requirement already up-to-date: enum34>=1.1.6 in /usr/local/envs/py2env/lib/python2.7/site-packages (from tensorflow)
    100% |████████████████████████████████| 7.4MB 78kB/s  eta 0:00:01
Requirement already up-to-date: absl-py>=0.1.6 in /usr/local/envs/py2env/lib/python2.7/site-packages (from tensorflow)
    100% |████████████████████████████████| 6.4MB 91kB/s  eta 0:00:01
Requirement already up-to-date: markdown>=2.6.8 in /usr

In [37]:
import datalab.bigquery as bq
import seaborn as sns
import pandas as pd
import numpy as np
import shutil
import tensorflow as tf
print tf.__version__
import os

1.5.0


<h1> Baseline model and compute its RSME </h1>

In [3]:
def estimate_stress(df):
  columns = [abs(df['ACCELERATION']), abs(df['PITCH']), abs(df['ROLL']), abs(df['YAW'])]
  stress_level = sum(columns)/4
  return stress_level

def compute_rmse(actual, predicted):
  return np.sqrt(np.mean((actual-predicted)**2))

def print_rmse(df, econsump, name):
  print ("{1} RMSE = {0}".format(compute_rmse(df['ECONSUMPTION'], econsump*estimate_stress(df)), name))

FEATURES = ['ACCELERATION', 'PITCH','ROLL','YAW']
TARGET = 'ECONSUMPTION'
columns = list([TARGET])
columns.extend(FEATURES) # in CSV, target is the first column, after the features
columns.append('key')
df_train = pd.read_csv('vehicles-train.csv', header=None, names=columns)
df_valid = pd.read_csv('vehicles-valid.csv', header=None, names=columns)
df_test = pd.read_csv('vehicles-test.csv', header=None, names=columns)
econsump = df_train['ECONSUMPTION'].mean() * estimate_stress(df_train).mean()/4
print ("averageEconsumption = ", econsump)
print_rmse(df_train, econsump, 'Train')
print_rmse(df_valid, econsump, 'Valid')
print_rmse(df_test, econsump, 'Test')

('averageEconsumption = ', 0.010274811520608038)
Train RMSE = 0.149345829532
Valid RMSE = 0.148660663726
Test RMSE = 0.146238892905


<h1> ML Regression with Neural Network </h1>

Now lets try and run a smarter model.

In [43]:
PROJECT = 'go-de-internal'
BUCKET = 'go-de-internal-johanna'
REGION = 'europe-west1'
REPO = "/content/datalab/demo-cases/vehilce-health-status"
os.listdir(REPO)

['.ipynb_checkpoints',
 'data-preparation.ipynb',
 'vehicles-valid.csv',
 'test.json',
 'vehicles-test.csv',
 'vehicles-train.csv',
 'econsumption',
 'linear-regression-with-neural-networks.ipynb']

In [44]:
# for bash
os.environ['PROJECT'] = PROJECT
os.environ['BUCKET'] = BUCKET
os.environ['REGION'] = REGION
os.environ['REPO'] = REPO

In [10]:
%bash
gcloud config set project $PROJECT
gcloud config set compute/region $REGION

Updated property [core/project].
Updated property [compute/region].


In [11]:
%bash
PROJECT_ID=$PROJECT
AUTH_TOKEN=$(gcloud auth print-access-token)
SVC_ACCOUNT=$(curl -X GET -H "Content-Type: application/json" \
    -H "Authorization: Bearer $AUTH_TOKEN" \
    https://ml.googleapis.com/v1/projects/${PROJECT_ID}:getConfig \
    | python -c "import json; import sys; response = json.load(sys.stdin); \
    print response['serviceAccount']")

echo "Authorizing the Cloud ML Service account $SVC_ACCOUNT to access files in $BUCKET"
gsutil -m defacl ch -u $SVC_ACCOUNT:R gs://$BUCKET
gsutil -m acl ch -u $SVC_ACCOUNT:R -r gs://$BUCKET  # error message (if bucket is empty) can be ignored
gsutil -m acl ch -u $SVC_ACCOUNT:W gs://$BUCKET

Authorizing the Cloud ML Service account service-462605511119@cloud-ml.google.com.iam.gserviceaccount.com to access files in go-de-internal-johanna


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   133    0   133    0     0    151      0 --:--:-- --:--:-- --:--:--   151
Updated default ACL on gs://go-de-internal-johanna/
Updated ACL on gs://go-de-internal-johanna/churn-prediction/
Updated ACL on gs://go-de-internal-johanna/churn-prediction/mv_ab_model_ads_build_sep.csv
Updated ACL on gs://go-de-internal-johanna/flight-time-estimate/
Updated ACL on gs://go-de-internal-johanna/vehilce-health-status/lindeshow-case-data-all.csv
Updated ACL on gs://go-de-internal-johanna/vehilce-health-status/
Updated ACL on gs://go-de-internal-johanna/pattern-recognition/
Updated ACL on gs://go-de-internal-johanna/


In [12]:
!cat econsumption/trainer/model.py

#!/usr/bin/env python

# Copyright 2017 Google Inc. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf
from tensorflow.contrib import layers
import tensorflow.contrib.learn as tflearn
from tensorflow.contrib import metrics


tf.logging.set_verbosity(tf.logging.INFO)

CSV_COLUMNS = ['ECONSUMPTION', 'ACCELERATION','PITCH','ROLL','YAW', 'key']

LABE

In [13]:
%bash
head -1 $REPO/vehicles-train.csv
head -1 $REPO/vehicles-valid.csv

0.0011634026758261544,0.020811654526534856,0.43367346938775514,0.3654822335025381,0.8838383838383839,0
0.16021716849948758,0.008324661810613941,0.3928571428571429,0.4974619289340102,0.4292929292929293,0


<h2> Running the Python module from the command-line </h2>

In [38]:
%bash
rm -rf vehicles.tar.gz vehicles_trained
export PYTHONPATH=${PYTHONPATH}:${REPO}/econsumption
python -m trainer.task \
   --train_data_paths="${REPO}/vehicles-train*" \
   --eval_data_paths=${REPO}/vehicles-valid.csv  \
   --output_dir=${REPO}/vehicles_trained \
   --num_epochs=10 \
   --job-dir=./tmp

/usr/local/envs/py2env/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_task_type': None, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f0571682690>, '_model_dir': '/content/datalab/demo-cases/vehilce-health-status/vehicles_trained/', '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_session_config': None, '_tf_random_seed': None, '_save_summary_steps': 100, '_environment': 'local', '_num_worker_replicas': 0, '_task_id': 0, '_log_step_count_steps': 100, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0

In [39]:
!ls $REPO/vehicles_trained/export/Servo

1520258144


In [40]:
%writefile ./test.json
{"ACCELERATION": 0.3424933,"PITCH": 0.4929889823,"ROLL": 0.999398298932,"YAW": 0.103000023}

Overwriting ./test.json


In [41]:
%bash
model_dir=$(ls ${REPO}/vehicles_trained/export/Servo)
gcloud ml-engine local predict \
    --model-dir=${REPO}/vehilces_trained/export/Servo/${model_dir} \
    --json-instances=./test.json

ERROR: (gcloud.ml-engine.local.predict) /usr/local/envs/py2env/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Traceback (most recent call last):
  File "lib/googlecloudsdk/command_lib/ml_engine/local_predict.py", line 111, in <module>
    main()
  File "lib/googlecloudsdk/command_lib/ml_engine/local_predict.py", line 106, in main
    instances=instances)
  File "/tools/google-cloud-sdk/lib/third_party/cloud_ml_engine_sdk/prediction/prediction_lib.py", line 1304, in local_predict
    client = create_client(framework, model_dir, tags)
  File "/tools/google-cloud-sdk/lib/third_party/cloud_ml_engine_sdk/prediction/prediction_lib.py", line 1242, in create_client
    return create_client_fn(model_path, tags)
  File "/tools/google-cloud-sdk/lib/third_

<h2> Running locally using gcloud </h2>

In [46]:
%bash
rm -rf vehicles.tar.gz vehicles_trained
gcloud ml-engine local train \
   --module-name=trainer.task \
   --package-path=${REPO}/econsumption/trainer \
   -- \
   --train_data_paths=${REPO}/vehicles-train.csv \
   --eval_data_paths=${REPO}/vehicles-valid.csv  \
   --num_epochs=10 \
   --output_dir=${REPO}/vehicles_trained 

/usr/local/envs/py2env/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_task_type': None, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f4195efcd50>, '_model_dir': '/content/datalab/demo-cases/vehilce-health-status/vehicles_trained/', '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_session_config': None, '_tf_random_seed': None, '_save_summary_steps': 100, '_environment': u'cloud', '_num_worker_replicas': 0, '_task_id': 0, '_log_step_count_steps': 100, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.

In [36]:
from google.datalab.ml import TensorBoard
TensorBoard().start('{}/vehicles_trained'.format(REPO))

TensorBoard was started successfully with pid 20213. Click here to access it.

20213

In [37]:
for pid in TensorBoard.list()['pid']:
  TensorBoard().stop(pid)
  print 'Stopped TensorBoard with pid {}'.format(pid)

Stopped TensorBoard with pid 20213


In [38]:
!ls $REPO/vehicles_trained

ls: cannot access '/content/datalab/democases-go-reply-internal/vehicle-health-status/vehicles_trained': No such file or directory


In [49]:
%bash
model_dir=$(ls ${REPO}/vehicles_trained/export/Servo)
gcloud ml-engine local predict \
    --model-dir=${REPO}/vehilces_trained/export/Servo/${model_dir} \
    --json-instances=./test.json

ERROR: (gcloud.ml-engine.local.predict) /usr/local/envs/py2env/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Traceback (most recent call last):
  File "lib/googlecloudsdk/command_lib/ml_engine/local_predict.py", line 111, in <module>
    main()
  File "lib/googlecloudsdk/command_lib/ml_engine/local_predict.py", line 106, in main
    instances=instances)
  File "/tools/google-cloud-sdk/lib/third_party/cloud_ml_engine_sdk/prediction/prediction_lib.py", line 1304, in local_predict
    client = create_client(framework, model_dir, tags)
  File "/tools/google-cloud-sdk/lib/third_party/cloud_ml_engine_sdk/prediction/prediction_lib.py", line 1242, in create_client
    return create_client_fn(model_path, tags)
  File "/tools/google-cloud-sdk/lib/third_

<h2> Submit training job using gcloud </h2>

First copy the training data to the cloud.  Then, launch a training job.

In [47]:
%bash
echo $BUCKET
gsutil -m rm -rf gs://${BUCKET}/vehilce-health-status/neural-net
gsutil -m cp ${REPO}/*.csv gs://${BUCKET}/vehilce-health-status//neural-net

go-de-internal-johanna


CommandException: 1 files/objects could not be removed.
Copying file:///content/datalab/demo-cases/vehilce-health-status/vehicles-test.csv [Content-Type=text/csv]...
Copying file:///content/datalab/demo-cases/vehilce-health-status/vehicles-train.csv [Content-Type=text/csv]...
Copying file:///content/datalab/demo-cases/vehilce-health-status/vehicles-valid.csv [Content-Type=text/csv]...
/ [3/3 files][  1.5 MiB/  1.5 MiB] 100% Done                                    
Operation completed over 3 objects/1.5 MiB.                                      


In [54]:
%%bash
OUTDIR=gs://${BUCKET}/vehilce-health-status/neural-net/vehicles_trained
JOBNAME=vehicles_health_demo_$(date -u +%y%m%d_%H%M%S)
echo $OUTDIR $REGION $JOBNAME
gsutil -m rm -rf $OUTDIR
gcloud ml-engine jobs submit training $JOBNAME \
   --region=$REGION \
   --module-name=trainer.task \
   --package-path=${REPO}/econsumption/trainer \
   --job-dir=$OUTDIR \
   --staging-bucket=gs://$BUCKET \
   --scale-tier=BASIC \
   --runtime-version=1.0 \
   -- \
   --train_data_paths="gs://${BUCKET}/econsumption/smallinput/vehicles-train*" \
   --eval_data_paths="gs://${BUCKET}/econsumption/smallinput/vehicles-valid*"  \
   --output_dir=$OUTDIR \
   --num_epochs=100

gs://go-de-internal-johanna/vehilce-health-status/neural-net/vehicles_trained europe-west1 vehicles_health_demo_180305_145424
jobId: vehicles_health_demo_180305_145424
state: QUEUED


CommandException: 1 files/objects could not be removed.
/tools/google-cloud-sdk/lib/googlecloudsdk/core/util/files.py:622: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  for chunk in iter(lambda: fp.read(4096), ''):
Job [vehicles_health_demo_180305_145424] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ml-engine jobs describe vehicles_health_demo_180305_145424

or continue streaming the logs with the command

  $ gcloud ml-engine jobs stream-logs vehicles_health_demo_180305_145424


<h1> Deploy model </h1>

Find out the actual name of the subdirectory where the model is stored and use it to deploy the model.

In [50]:
%bash
gsutil ls gs://${BUCKET}/econsumption/smallinput/vehicles_trained/export/Servo

CommandException: One or more URLs matched no objects.


In [42]:
%bash
MODEL_NAME="econsumption"
MODEL_VERSION="v1"
MODEL_LOCATION=$(gsutil ls gs://${BUCKET}/econsumption/smallinput/vehicles_trained/export/Servo | tail -1)
echo "Deleting and deploying $MODEL_NAME $MODEL_VERSION from $MODEL_LOCATION ... this will take a few minutes"
#gcloud ml-engine versions delete ${MODEL_VERSION} --model ${MODEL_NAME}
#gcloud ml-engine models delete ${MODEL_NAME}
gcloud ml-engine models create ${MODEL_NAME} --regions $REGION
gcloud ml-engine versions create ${MODEL_VERSION} --model ${MODEL_NAME} --origin ${MODEL_LOCATION}

Deleting and deploying econsumption v1 from  ... this will take a few minutes


CommandException: One or more URLs matched no objects.
Created ml engine model [projects/go-de-internal/models/econsumption].
ERROR: (gcloud.ml-engine.versions.create) argument --origin: expected one argument
Usage: gcloud ml-engine versions create VERSION --model=MODEL [optional flags]
  optional flags may be  --async | --config | --description | --help |
                         --labels | --origin | --runtime-version |
                         --staging-bucket

For detailed information on this command and its flags, run:
  gcloud ml-engine versions create --help


<h1> Prediction </h1>

In [43]:
%bash
gcloud ml-engine predict --model=econsumption --version=v1 --json-instances=./test.json

ERROR: (gcloud.ml-engine.predict) HTTP request failed. Response: {
  "error": {
    "code": 404,
    "message": "Field: name Error: The specified model version was not found.",
    "status": "NOT_FOUND",
    "details": [
      {
        "@type": "type.googleapis.com/google.rpc.BadRequest",
        "fieldViolations": [
          {
            "field": "name",
            "description": "The specified model version was not found."
          }
        ]
      }
    ]
  }
}



In [44]:
from googleapiclient import discovery
from oauth2client.client import GoogleCredentials
import json

credentials = GoogleCredentials.get_application_default()
api = discovery.build('ml', 'v1', credentials=credentials,
            discoveryServiceUrl='https://storage.googleapis.com/cloud-ml/discovery/ml_v1_discovery.json')

request_data = {'instances':
  [
      {
        'ACCELERATION': 0.3424933,
        'PITCH': 0.49298898232,
        'ROLL': 0.999398298932,
        'YAW': 0.103000023,
      }
  ]
}

parent = 'projects/%s/models/%s/versions/%s' % (PROJECT, 'econsumption', 'v1')
response = api.projects().predict(body=request_data, name=parent).execute()
print "response={0}".format(response)

HttpError: <HttpError 404 when requesting https://ml.googleapis.com/v1/projects/go-de-internal/models/econsumption/versions/v1:predict?alt=json returned "Field: name Error: The specified model version was not found.">

<h1> Create hyper-parameter tuning </h1>

The file specifies the search region in parameter space.  Cloud MLE carries out a smart search algorithm within these constraints (i.e. it does not try out every single value).

In [51]:
!grep -A 2 add_argument econsumption/trainer/task.py

  parser.add_argument(
      '--train_data_paths',
      help='GCS or local path to training data',
--
  parser.add_argument(
      '--num_epochs',
      help="""\
--
  parser.add_argument(
      '--train_batch_size',
      help='Batch size for training steps',
--
  parser.add_argument(
      '--eval_batch_size',
      help='Batch size for evaluation steps',
--
  parser.add_argument(
      '--train_steps',
      help="""\
--
  parser.add_argument(
      '--eval_steps',
      help='Number of steps to run evalution for at each checkpoint',
--
  parser.add_argument(
      '--eval_data_paths',
      help='GCS or local path to evaluation data',
--
  parser.add_argument(
      '--hidden_units',
      help='List of hidden layer sizes to use for DNN feature columns',
--
  parser.add_argument(
      '--output_dir',
      help='GCS location to write checkpoints and export models',
--
  parser.add_argument(
      '--job-dir',
      help='this model ignores this field, but it is required by gcloud

In [52]:
!grep -A 5 eval_metric econsumption/trainer/model.py

def get_eval_metrics():
  return {
     'rmse': tflearn.MetricSpec(metric_fn=metrics.streaming_root_mean_squared_error),
  }


In [53]:
%writefile hyperparam.yaml
trainingInput:
  scaleTier: STANDARD_1
  hyperparameters:
    goal: MINIMIZE
    maxTrials: 30
    maxParallelTrials: 1
    params:
    - parameterName: train_batch_size
      type: INTEGER
      minValue: 64
      maxValue: 512
      scaleType: UNIT_LOG_SCALE
    - parameterName: nbuckets
      type: INTEGER
      minValue: 10
      maxValue: 20
      scaleType: UNIT_LINEAR_SCALE
    - parameterName: hidden_units
      type: CATEGORICAL
      categoricalValues: ["128 32", "256 128 16", "64 64 64 8"]       

Writing hyperparam.yaml


In [55]:
%bash
echo $BUCKET
gsutil -m rm -rf gs://${BUCKET}/vehilce-health-status/hyper-param
gsutil -m cp ${REPO}/*.csv gs://${BUCKET}/vehilce-health-status/hyper-param/

go-de-internal-johanna


CommandException: 1 files/objects could not be removed.
Copying file:///content/datalab/demo-cases/vehilce-health-status/vehicles-test.csv [Content-Type=text/csv]...
Copying file:///content/datalab/demo-cases/vehilce-health-status/vehicles-train.csv [Content-Type=text/csv]...
Copying file:///content/datalab/demo-cases/vehilce-health-status/vehicles-valid.csv [Content-Type=text/csv]...
/ [3/3 files][  1.5 MiB/  1.5 MiB] 100% Done                                    
Operation completed over 3 objects/1.5 MiB.                                      


In [49]:
%%bash
OUTDIR=gs://${BUCKET}/vehilce-health-status/hyper-param/vehicles_trained
JOBNAME=vehicles_health_demo_hyper_param_$(date -u +%y%m%d_%H%M%S)
echo $OUTDIR $REGION $JOBNAME
gsutil -m rm -rf $OUTDIR
gcloud ml-engine jobs submit training $JOBNAME \
   --region=$REGION \
   --module-name=trainer.task \
   --package-path=${REPO}/econsumption/trainer \
   --job-dir=$OUTDIR \
   --staging-bucket=gs://$BUCKET \
   --scale-tier=STANDARD_1 \
   --runtime-version=1.2 \
   --config=hyperparam.yaml \
   -- \
   --train_data_paths="gs://$BUCKET/vehilce-health-status/hyper-param/vehicles-train*" \
   --eval_data_paths="gs://${BUCKET}/vehilce-health-status/hyper-param/vehicles-valid*"  \
   --output_dir=$OUTDIR \
   --num_epochs=100

gs://storage-democases-go-reply-internal/econsumption/hyper_param/vehicles_trained us-central1 vehicles_health_demo_hyper_param_180226_153810
jobId: vehicles_health_demo_hyper_param_180226_153810
state: QUEUED


CommandException: 1 files/objects could not be removed.
/tools/google-cloud-sdk/lib/googlecloudsdk/core/util/files.py:622: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  for chunk in iter(lambda: fp.read(4096), ''):
Job [vehicles_health_demo_hyper_param_180226_153810] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ml-engine jobs describe vehicles_health_demo_hyper_param_180226_153810

or continue streaming the logs with the command

  $ gcloud ml-engine jobs stream-logs vehicles_health_demo_hyper_param_180226_153810
